In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

rng = np.random.default_rng(7)
idx = pd.period_range("2018-01", periods=60, freq="M")
trend = 0.2 * np.arange(len(idx))
season = 2 * np.sin(2 * np.pi * np.arange(len(idx)) / 12)
noise = rng.normal(scale=0.8, size=len(idx))

y = pd.Series(trend + season + noise, index=idx, name="y")


In [2]:
fig = px.line(y.to_timestamp(), title="Univariate Series (pd.Series)")
fig.update_layout(xaxis_title="Time", yaxis_title="Value")
fig.show()


In [3]:
ids = ["A", "B", "C"]
panel = pd.DataFrame(
    {
        "id": np.repeat(ids, len(idx)),
        "time": np.tile(idx, len(ids)),
        "value": np.tile(y.values, len(ids)) + rng.normal(scale=0.6, size=len(ids) * len(idx)),
    }
).set_index(["id", "time"])

panel_long = panel.reset_index()
fig = px.line(
    panel_long,
    x=panel_long["time"].dt.to_timestamp(),
    y="value",
    color="id",
    title="Panel Data (MultiIndex: id, time)",
)
fig.update_layout(xaxis_title="Time", yaxis_title="Value")
fig.show()


In [4]:
groups = ["North", "South"]
stores = ["S1", "S2"]
index = pd.MultiIndex.from_product([groups, stores, idx], names=["region", "store", "time"])

values = []
for g in groups:
    for s in stores:
        g_effect = 2.0 if g == "North" else -1.0
        s_effect = 0.5 if s == "S1" else -0.3
        values.append(y.values + g_effect + s_effect + rng.normal(scale=0.4, size=len(idx)))

hier = pd.Series(np.concatenate(values), index=index, name="sales")

# Aggregate to region level for visualization
region = hier.groupby(level=["region", "time"]).sum().reset_index()
fig = px.line(
    region,
    x=region["time"].dt.to_timestamp(),
    y="sales",
    color="region",
    title="Hierarchical Series (region → store → time)",
)
fig.update_layout(xaxis_title="Time", yaxis_title="Sales")
fig.show()


In [5]:
X = pd.DataFrame(
    {
        "promo": (rng.random(len(idx)) > 0.8).astype(int),
        "price": 10 + rng.normal(scale=0.3, size=len(idx)),
    },
    index=idx,
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=y.index.to_timestamp(), y=y, name="y"))
fig.add_trace(go.Scatter(x=X.index.to_timestamp(), y=X["price"], name="price", yaxis="y2"))
fig.update_layout(
    title="Target + Exogenous Feature",
    xaxis_title="Time",
    yaxis=dict(title="y"),
    yaxis2=dict(title="price", overlaying="y", side="right"),
)
fig.show()
